# Window generator class

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,train_df, val_df, test_df,label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
    self.column_indices = {name: i for i, name in enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self): #return the string representation of the object
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])
  
  def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    if self.label_columns is not None:
        labels = tf.stack(
            [labels[:, :, self.column_indices[name]] for name in self.label_columns],
            axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])
    return inputs, labels
  
  def plot(self, model=None, plot_col='TD (degC)', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
      plt.subplot(max_n, 1, n+1)
      plt.ylabel(f'{plot_col} [normed]')
      plt.plot(self.input_indices, inputs[n, :, plot_col_index],
              label='Inputs', marker='.', zorder=-10)

      if self.label_columns:
        label_col_index = self.label_columns_indices.get(plot_col, None)
      else:
        label_col_index = plot_col_index

      if label_col_index is None:
        continue

      plt.scatter(self.label_indices, labels[n, :, label_col_index],
                  edgecolors='k', label='Labels', c='#2ca02c', s=64)
      if model is not None:
        predictions = model(inputs)
        plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                    marker='X', edgecolors='k', label='Predictions',
                    c='#ff7f0e', s=64)

      if n == 0:
        plt.legend()
      
      plt.xlabel('Time [h]')


  def make_dataset(self, data):
    data = np.array(data, dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(
        data=data,
        targets=None,
        sequence_length=self.total_window_size,
        sequence_stride=1,
        shuffle=True,
        batch_size=32,)

    ds = ds.map(self.split_window)

    return ds
  
  @property #decorator to make a method behave like an attribute of the class
  def train(self): 
    return self.make_dataset(self.train_df)

  @property
  def val(self):
    return self.make_dataset(self.val_df)

  @property
  def test(self):
    return self.make_dataset(self.test_df)

  @property
  def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
      # No example batch was found, so get one from the `.train` dataset
      result = next(iter(self.train))
      # And cache it for next time
      self._example = result
    return result

# Understanding step by step tutorial

## Step 1 - working only with window generator class

In [5]:
#!/usr/bin/env python

import pandas as pd
import numpy as np
import tensorflow as tf

def main():
    # 1) Create some dummy DataFrames as if they are your train/val/test sets.
    #    We'll create random data with columns: ["Rain", "TD (degC)", "RH"]
    num_samples = 1000
    columns = ["Rain", "TD (degC)", "RH"]

    # Make random data for each column
    data = np.random.rand(num_samples, len(columns)).astype(np.float32)

    # Convert to DataFrame
    full_df = pd.DataFrame(data, columns=columns)

    # Let's split into train/val/test
    train_df = full_df.iloc[:700]
    val_df   = full_df.iloc[700:850]
    test_df  = full_df.iloc[850:]

    # 2) Choose window parameters
    input_width = 24  # e.g. last 24 timesteps for input
    label_width = 1   # e.g. predict 1 step ahead
    shift = 1         # that next step is 1 step after the input

    # We want to predict the column "TD (degC)" only
    label_columns = ["TD (degC)"]

    # 3) Instantiate the WindowGenerator
    w = WindowGenerator(input_width=input_width,
                        label_width=label_width,
                        shift=shift,
                        train_df=train_df,
                        val_df=val_df,
                        test_df=test_df,
                        label_columns=label_columns)

    # 4) Print the window configuration
    print(w)

    # 5) Access a batch from the training dataset
    example_inputs, example_labels = next(iter(w.train))
    print("Example input batch shape:", example_inputs.shape)
    print("first: The batch size, second: The length of the input sequence for each example, third: The number of features (columns) in your data.")

    print("Example label batch shape:", example_labels.shape)

    # 6) (Optional) If you want to see how the first sample in that batch looks:
    print("First sample input:", example_inputs[0])
    print("First sample label:", example_labels[0])

if __name__ == "__main__":
    main()


Total window size: 25
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24]
Label column name(s): ['TD (degC)']
Example input batch shape: (32, 24, 3)
first: The batch size, second: The length of the input sequence for each example, third: The number of features (columns) in your data.
Example label batch shape: (32, 1, 1)
First sample input: tf.Tensor(
[[4.0168700e-01 5.0459886e-01 8.2421970e-01]
 [3.9037362e-01 7.5998867e-01 4.9404573e-02]
 [2.4878137e-01 1.8084264e-01 2.7639589e-01]
 [5.3209162e-01 2.3457150e-01 3.4356412e-01]
 [9.7648728e-01 4.8141509e-01 7.9466629e-01]
 [7.7820696e-02 2.1349562e-02 6.5592892e-02]
 [5.3392988e-01 5.8770972e-01 4.5982581e-01]
 [2.3215547e-01 5.8226188e-04 8.0550975e-01]
 [3.9349687e-01 5.3515112e-01 3.6645806e-01]
 [7.4941236e-01 4.0955618e-01 8.3430678e-01]
 [9.8743916e-01 9.9407321e-01 8.1653047e-01]
 [8.7993968e-01 3.0999726e-01 2.2300580e-01]
 [8.6759456e-02 4.9507588e-01 7.8161466e-01]
 [8.

# step 2 - adding cnn per station

In [6]:
#!/usr/bin/env python
# train_individual_cnn.py

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.models import Sequential

def build_cnn_for_station(input_width, num_features, station_id=None):
    """
    Build a CNN branch for a single station.
    This branch accepts input of shape (input_width, num_features)
    and outputs an embedding vector (here, of length 32).

    The branch's layers will have unique names if station_id is provided.
    """
    name_suffix = f"_{station_id}" if station_id is not None else ""
    inp = Input(shape=(input_width, num_features), name=f"station_input{name_suffix}")
    # A simple 1D CNN: a Conv1D with 'causal' padding followed by global pooling.
    x = layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='causal',
                      name=f"conv1d{name_suffix}")(inp)
    x = layers.GlobalAveragePooling1D(name=f"global_avg_pool{name_suffix}")(x)
    return Model(inputs=inp, outputs=x, name=f"station_cnn{name_suffix}")

def build_individual_cnn_model(input_width, num_stations, num_features):
    """
    Build a model that:
      - Accepts input with shape (input_width, num_stations, num_features)
      - For each station:
          * Slices the input for that station (resulting shape: (input_width, num_features))
          * Applies an individual (not shared) CNN branch (with unique weights)
      - Concatenates the station-specific embeddings and predicts a single output.
    """
    # Define the overall input.
    inputs = Input(shape=(input_width, num_stations, num_features), name="input_window")
    
    # List to hold the outputs from each station's CNN branch.
    station_outputs = []
    
    # Loop over the station indices and build an independent branch for each.
    for i in range(num_stations):
        # Use a Lambda layer to extract station i (slicing along the station axis).
        station_i = layers.Lambda(lambda x, idx=i: x[:, :, idx, :],
                                  name=f"slice_station_{i}")(inputs)
        # Now, station_i has shape (batch, input_width, num_features)
        
        # Build a separate CNN branch for station i with a unique name.
        cnn_branch = build_cnn_for_station(input_width, num_features, station_id=i)
        # Get the CNN output (embedding) for this station.
        branch_output = cnn_branch(station_i)
        station_outputs.append(branch_output)
    
    # Combine (concatenate) all station embeddings into one vector.
    combined = layers.concatenate(station_outputs, axis=-1)  # shape: (batch, num_stations * 32)
    
    # Optional: add some fully connected layers here if desired.
    outputs = layers.Dense(1, name="forecast")(combined)
    
    model = Model(inputs=inputs, outputs=outputs, name="IndividualCNNPerStation")
    model.compile(optimizer='adam', loss='mse')
    return model

def main():
    # Set model parameters:
    input_width = 24      # e.g., 24 timesteps per window
    num_stations = 5      # e.g., 5 stations
    num_features = 3      # features per station (e.g., "Rain", "TD (degC)", "RH")
    
    # Build the model.
    model = build_individual_cnn_model(input_width, num_stations, num_features)
    model.summary()
    
    # Generate some dummy data:
    # Create 1000 samples; each sample shape = (24, 5, 3)
    num_samples = 1000
    X = np.random.rand(num_samples, input_width, num_stations, num_features).astype(np.float32)
    y = np.random.rand(num_samples, 1).astype(np.float32)
    
    # Train the model.
    model.fit(X, y, epochs=3, batch_size=32, validation_split=0.1)
    
    # Save the model (using the native Keras format).
    model.save("cnn_individual_station_model.keras")
    print("Model saved to 'cnn_individual_station_model.keras'.")

if __name__ == "__main__":
    main()


ValueError: The name "station_cnn" is used 5 times in the model. All operation names should be unique.